Custom dataset loading

In [2]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms

# Define the transformations for the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset
train_dataset = ImageFolder(root='../dataset/train', transform=transform)
val_dataset = ImageFolder(root='../dataset/val', transform=transform)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)

/home/joey/M1/ouverture_recherche_2024/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


You can create a custom DataModule class to handle the dataset loading and preprocessing. This class will be used by the Trainer to manage the data.

In [3]:
from pytorch_lightning import LightningDataModule

class SolderingDefectDataModule(LightningDataModule):
    def __init__(self, train_dataset, val_dataset, batch_size=32, num_workers=8):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size
        self.num_workers = num_workers

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

# Initialize the DataModule
datamodule = SolderingDefectDataModule(train_dataset, val_dataset)

You can use the same FastFlow model as in the original notebook. The model will be trained to detect anomalies in the soldering defects.

In [7]:
from anomalib.models import Patchcore

model = Patchcore()

INFO:anomalib.models.components.base.anomaly_module:Initializing Patchcore model.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/wide_resnet50_2.racm_in1k)
INFO:timm.models._hub:[timm/wide_resnet50_2.racm_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


Optimizer and callbacks

In [16]:
from pytorch_lightning import LightningModule
from torchmetrics import AUROC

class MyLightningModule(LightningModule):
    def __init__(self):
        super().__init__()
        self.image_auroc = AUROC(task="binary")  # Image-level AUROC
        self.pixel_auroc = AUROC(task="binary")  # Pixel-level AUROC

    def validation_step(self, batch, batch_idx):
        # Compute predictions
        preds = self(batch["image"])
        # Update metrics
        self.image_auroc(preds["image_scores"], batch["labels"])
        self.pixel_auroc(preds["anomaly_maps"].flatten(), batch["mask"].flatten())

    def on_validation_epoch_end(self):
        # Log metrics
        self.log("image_AUROC", self.image_auroc.compute(), prog_bar=True)
        self.log("pixel_AUROC", self.pixel_auroc.compute(), prog_bar=True)
        # Reset metrics
        self.image_auroc.reset()
        self.pixel_auroc.reset()

In [24]:
from pytorch_lightning import Trainer

trainer = Trainer(
    callbacks=callbacks,
    accelerator="auto",  # <"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    max_epochs=100,
)

trainer.fit(model=model, datamodule=datamodule)

INFO:anomalib.models.components.base.anomaly_module:Initializing Patchcore model.


INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
INFO:timm.models._hub:[timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type        | Params | Mode 
----------------------------------------------------
0 | image_auroc | BinaryAUROC | 0      | train
1 | pixel_auroc | BinaryAUROC | 0      | train
2 | model       | Patchcore   | 2.8 M  | train
----------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated model params size (MB)
7         M

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)